In [ ]:
from pyspark.sql import SparkSession
import pandas as pd
from sqlalchemy import create_engine


In [3]:
# Initialize the Spark session
spark = SparkSession.builder.appName("NugaBankETL").getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
26/01/17 23:47:21 WARN Utils: Your hostname, Davids-MacBook-Air-2.local, resolves to a loopback address: 127.0.0.1; using 192.168.40.9 instead (on interface en0)
26/01/17 23:47:21 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/01/17 23:47:22 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
spark


In [5]:
nuga_bank_df = spark.read.csv(r'path/to/nuga_bank_data.csv', header=True, inferSchema=True)
nuga_bank_df.show(5)

26/01/18 00:30:36 WARN FileStreamSink: Assume no metadata directory. Error while looking for metadata directory in the path: path/to/nuga_bank_data.csv.
java.io.FileNotFoundException: File path/to/nuga_bank_data.csv does not exist
	at org.apache.hadoop.fs.RawLocalFileSystem.deprecatedGetFileStatus(RawLocalFileSystem.java:980)
	at org.apache.hadoop.fs.RawLocalFileSystem.getFileLinkStatusInternal(RawLocalFileSystem.java:1301)
	at org.apache.hadoop.fs.RawLocalFileSystem.getFileStatus(RawLocalFileSystem.java:970)
	at org.apache.hadoop.fs.FilterFileSystem.getFileStatus(FilterFileSystem.java:462)
	at org.apache.spark.sql.execution.streaming.sinks.FileStreamSink$.hasMetadata(FileStreamSink.scala:58)
	at org.apache.spark.sql.execution.datasources.DataSource.resolveRelation(DataSource.scala:384)
	at org.apache.spark.sql.catalyst.analysis.ResolveDataSource.org$apache$spark$sql$catalyst$analysis$ResolveDataSource$$loadV1BatchSource(ResolveDataSource.scala:143)
	at org.apache.spark.sql.catalyst.an

AnalysisException: [PATH_NOT_FOUND] Path does not exist: file:/Users/davidabejide/Documents/GitHub/Nuga_bankz/path/to/nuga_bank_data.csv. SQLSTATE: 42K03